In [3]:
from DbConnector import DbConnector
#from part2 import Database
from tabulate import tabulate

# Set up the program
from dotenv import load_dotenv
import os
import numpy as np
from datetime import datetime, timedelta
from haversine import haversine, Unit

load_dotenv()

db = DbConnector()

You are connected to the database: assignment3
-----------------------------------------------



### Task 1

How many users, activities and trackpoints are there in the dataset (after it is inserted into the database)

### Task 2
Find the average number of activities per user.

### Task 3

Find the top 20 users with the highest number of activities.

In [ ]:
collection = connector.db["Activity"]

rows = collection.aggregate(
    [
        {"$group": {"_id":  "$user_id", "count": {"$sum": 1}}},
        {"$sort": {"count": -1}},
        {"$limit": 20},
    ]
)

for row in rows:
    pprint(row)

### Task 4
Find all users who have taken a taxi. 

### Task 5
Find all types of transportation modes and count how many activities that are tagged with these transportation mode labels. Do not count the rows where the mode is null.

### Task 6
a) Find the year with the most activities.

b) Is this also the year with most recorded hours?

### Task 7
Find the total distance (in km) walked in 2008, by user with id=112.

### Task 8

 Find the top 20 users who have gained the most altitude meters

### Task 9
Find all users who have invalid activities, and the number of invalid activities per user

### Task 10
Find the users who have tracked an activity in the Forbidden City of Beijing.

### Task 11
Find all users who have registered transportation_mode and their most used transportation_mode. 

In [ ]:
collection = connector.db["Activity"]

rows = collection.aggregate(
    [
        {"$match": {"transportation_mode": {"$ne": None}}},
        {
            "$group": {
                "_id": {
                    "user_id": "$user_id",
                    "transportation_mode": "$transportation_mode",
                },
                "count": {"$sum": 1},
            }
        },
        {
            "$lookup": {
                "from": "User",
                "let": {
                    "a_id": "$_id",
                    "a_has_labels": "$has_labels",
                },
                "pipeline": [{"$match": {"$expr": {"$eq": ["$$a_id", "$user_id"]}}}],
                "as": "saves_transportation",
            }
        },
        {"$match": {"saves_transportation": {"$ne": {"$a_has_labels": False}}}},
        {"$sort": {"_id.user_id": 1, "count": -1}},
        {
            "$group": {
                "_id": "$_id.user_id",
                "most_used_transportation_mode": {"$first": "$_id.transportation_mode"},
            }
        },
        {"$sort": {"_id.user_id": 1}},
    ]
)

for row in rows:
    pprint(row)